In [1]:
import sys
import os
sys.path.append(r"..")

from utils import *
import pandas as pd
import numpy as np

path_build = "../../data/tencent2020/build/"
path_save = "../../data/tencent2020/save/"

pickle_path = "../../data/tencent2020/pickle"
if not os.path.exists(pickle_path):
    os.mkdir(pickle_path) 
pd.set_option('display.max_columns', None)

In [2]:
train_click_log = pd.read_csv(path_save + "train_click_log.csv", encoding="utf-8")
test_click_log = pd.read_csv(path_save + "test_click_log.csv", encoding="utf-8")
train_click_log.shape, test_click_log.shape

((30082771, 9), (33585512, 9))

In [3]:
np.percentile(train_click_log['click_times'], 99.999),np.percentile(test_click_log['click_times'], 99.999),

(13.0, 25.0)

In [4]:
#经过验证，去除某些点击次数过高的数据，不会去除用户
aa = test_click_log[test_click_log['click_times']<=8]
aa.groupby('user_id').size().shape

(1000000,)

train：user_id 范围 (1, 900000)  
test ：user_id 范围 (3000001, 4000000) 

In [5]:
train_click_log.head()

,time,user_id,creative_id,click_times,ad_id,product_id,product_category,advertiser_id,industry
0,9,30920,567330,1,504423,30673,3,32638,319
1,65,30920,3072255,1,2642300,1261,2,6783,6
2,56,30920,2361327,1,2035918,1261,2,6783,6
3,6,309204,325532,1,292523,27081,3,32066,242
4,59,309204,2746730,1,2362208,0,18,14682,88


In [6]:
train_click_log.describe()

,time,user_id,creative_id,click_times,ad_id,product_id,product_category,advertiser_id,industry
count,3.008277e+07,3.008277e+07,3.008277e+07,3.008277e+07,3.008277e+07,3.008277e+07,3.008277e+07,3.008277e+07,3.008277e+07
mean,4.801738e+01,4.498556e+05,1.676532e+06,1.063127e+00,1.447917e+06,3.928835e+03,8.128011e+00,2.119445e+04,1.610350e+02
std,2.592504e+01,2.599939e+05,1.308054e+06,2.982831e-01,1.120569e+06,9.123197e+03,7.055121e+00,1.406527e+04,1.290624e+02
min,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,0.000000e+00,1.000000e+00,2.000000e+00,0.000000e+00
25%,2.600000e+01,2.246130e+05,4.158100e+05,1.000000e+00,3.711100e+05,0.000000e+00,2.000000e+00,1.098600e+04,8.000000e+00
50%,4.900000e+01,4.498040e+05,1.508864e+06,1.000000e+00,1.310317e+06,1.290000e+02,3.000000e+00,1.810300e+04,2.020000e+02
75%,7.100000e+01,6.752420e+05,2.740464e+06,1.000000e+00,2.355111e+06,1.873000e+03,1.800000e+01,3.003400e+04,2.930000e+02
max,9.100000e+01,9.000000e+05,4.445718e+06,1.520000e+02,3.812200e+06,4.431300e+04,1.800000e+01,6.296500e+04,3.350000e+02


In [7]:
test_click_log.head()

,time,user_id,creative_id,click_times,ad_id,product_id,product_category,advertiser_id,industry
0,20,3131989,645764,1,573314,58,2,14689,6
1,20,3131989,1027422,1,902764,129,2,42272,6
2,20,3131989,1106443,1,970829,2171,2,37513,322
3,20,3131989,629802,1,559183,0,18,14678,26
4,59,3131989,2839769,1,2441288,129,2,35328,6


In [8]:
test_click_log.describe()

,time,user_id,creative_id,click_times,ad_id,product_id,product_category,advertiser_id,industry
count,3.358551e+07,3.358551e+07,3.358551e+07,3.358551e+07,3.358551e+07,3.358551e+07,3.358551e+07,3.358551e+07,3.358551e+07
mean,4.797323e+01,3.499714e+06,1.674632e+06,1.066196e+00,1.446292e+06,3.922139e+03,8.125964e+00,2.122169e+04,1.611937e+02
std,2.593779e+01,2.883313e+05,1.308090e+06,3.561287e-01,1.120606e+06,9.098822e+03,7.055398e+00,1.408001e+04,1.290848e+02
min,1.000000e+00,3.000001e+06,1.000000e+00,1.000000e+00,1.000000e+00,0.000000e+00,1.000000e+00,1.000000e+00,0.000000e+00
25%,2.600000e+01,3.249486e+06,4.158050e+05,1.000000e+00,3.708990e+05,0.000000e+00,2.000000e+00,1.098600e+04,8.000000e+00
50%,4.900000e+01,3.500907e+06,1.506880e+06,1.000000e+00,1.309499e+06,1.290000e+02,3.000000e+00,1.810300e+04,2.030000e+02
75%,7.100000e+01,3.748320e+06,2.737762e+06,1.000000e+00,2.353237e+06,1.873000e+03,1.800000e+01,3.012200e+04,2.930000e+02
max,9.100000e+01,4.000000e+06,4.445720e+06,1.850000e+02,3.812202e+06,4.431400e+04,1.800000e+01,6.296500e+04,3.350000e+02


In [3]:
click_log = train_click_log.append(test_click_log)
click_log.sort_values(by=['user_id','time'], ascending=[True,True], inplace=True)
click_log = click_log.reset_index(drop=True)
click_log = click_log[click_log['click_times']<=8] #去除点击次数99.99%外的数据记录
print(click_log.shape)
click_log.head()

(63663353, 9)


,time,user_id,creative_id,click_times,ad_id,product_id,product_category,advertiser_id,industry
0,20,1,821396,1,724607,0,5,7293,326
1,20,1,209778,1,188507,136,2,9702,6
2,20,1,877468,1,773445,0,5,29455,106
3,39,1,1683713,1,1458878,0,5,14668,326
4,40,1,122032,1,109959,1334,2,11411,0


In [6]:
click_log = multi_column_LabelEncoder(click_log, columns=['creative_id','ad_id', 'product_id', 'product_category', 'advertiser_id',
                                                         'industry'])
print(click_log['creative_id'].nunique())
print(click_log['ad_id'].nunique())
print(click_log['product_id'].nunique())
print(click_log['product_category'].nunique())
print(click_log['advertiser_id'].nunique())
print(click_log['industry'].nunique())

LabelEncoder Successfully!
3412767
3027358
39057
18
57870
332


In [7]:
def ca_list(x_l):
    l = list(x_l)
    t = list(set(l))
    t.sort(key=l.index)
    return t
user_ids = click_log.groupby(['user_id']).agg({'creative_id':lambda x:list(x),
                                                        'ad_id':lambda x:list(x),
                                                        'product_id':lambda x:list(x),
                                                        'product_category':lambda x:list(x),
                                                        'advertiser_id':lambda x:list(x),
                                                        'industry':lambda x:list(x),   
                                                        'click_times':lambda x:list(x),
                                                         'time':lambda x:list(x),
                                                       }).reset_index()


In [8]:
user_ids['creative_id_len'] = user_ids['creative_id'].map(lambda x: len(set(x)))
user_ids['ad_id_len'] = user_ids['ad_id'].map(lambda x: len(set(x)))
user_ids['product_id_len'] = user_ids['product_id'].map(lambda x: len(set(x)))
user_ids['product_category_len'] = user_ids['product_category'].map(lambda x: len(set(x)))
user_ids['advertiser_len'] = user_ids['advertiser_id'].map(lambda x: len(set(x)))
user_ids['industry_len'] = user_ids['industry'].map(lambda x: len(set(x)))


In [9]:
user_ids.to_pickle(f"{pickle_path}/user_ids_relencode.pickle")
user_ids.head()

,user_id,creative_id,ad_id,product_id,product_category,advertiser_id,industry,click_times,time,creative_id_len,ad_id_len,product_id_len,product_category_len,advertiser_len,industry_len
0,1,"[642275, 170988, 685687, 1307085, 100774, 5953...","[581171, 156554, 620126, 1166887, 92441, 55685...","[1, 125, 1, 1, 1132, 1, 1, 1, 1, 1244, 55, 55,...","[5, 2, 5, 5, 2, 18, 5, 5, 18, 2, 2, 2, 2]","[6411, 8522, 26781, 12996, 10113, 13007, 15341...","[323, 7, 104, 323, 1, 323, 73, 214, 64, 235, 2...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2]","[20, 20, 20, 39, 40, 43, 46, 52, 60, 64, 64, 7...",12,12,6,3,12,9
1,2,"[52634, 128035, 32734, 478744, 10372, 349730, ...","[49388, 117016, 31581, 435462, 11432, 318236, ...","[76, 69, 118, 118, 1193, 76, 1, 1064, 1, 1, 10...","[2, 2, 2, 2, 2, 2, 18, 2, 18, 18, 2, 18, 2, 18...","[20681, 9438, 16619, 23545, 705, 20681, 31504,...","[315, 235, 7, 7, 314, 315, 47, 7, 47, 47, 239,...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","[10, 11, 14, 17, 28, 28, 28, 38, 38, 39, 41, 4...",42,42,20,3,36,15
2,3,"[518448, 632295, 556265, 645498, 466848, 56861...","[470940, 572190, 504577, 584182, 424822, 51556...","[31600, 35900, 1444, 31352, 1444, 1, 1, 857, 1...","[17, 17, 2, 17, 2, 18, 18, 2, 2, 2, 18, 12, 2,...","[30056, 8682, 16554, 12593, 15183, 7955, 7362,...","[1, 1, 319, 1, 319, 7, 54, 7, 319, 319, 202, 2...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","[12, 13, 14, 14, 14, 17, 19, 22, 31, 36, 37, 4...",30,30,17,6,28,8
3,4,"[32621, 464022, 452725, 1466785, 1520025, 1750...","[31491, 422207, 412076, 1308931, 1355934, 1558...","[1620, 1, 2363, 33897, 36230, 32982, 34972, 34...","[2, 18, 2, 17, 17, 4, 17, 17, 18, 2, 2, 2, 2, ...","[17456, 7015, 11589, 10744, 21413, 745, 8954, ...","[235, 26, 245, 1, 1, 1, 1, 1, 86, 316, 235, 7,...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","[8, 15, 41, 44, 48, 48, 48, 48, 49, 52, 58, 58...",29,29,18,6,26,10
4,5,"[238161, 280415, 19832, 35721, 666206, 821172,...","[218107, 256798, 20328, 34244, 602347, 740737,...","[1, 1, 76, 125, 1, 125, 1, 1, 1, 889, 1, 2358,...","[5, 8, 2, 2, 18, 2, 5, 18, 18, 2, 18, 2, 5, 5,...","[10525, 923, 20681, 8526, 35446, 2611, 15865, ...","[294, 1, 315, 7, 319, 7, 285, 319, 316, 235, 3...","[1, 1, 1, 1, 1, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","[3, 13, 14, 15, 20, 21, 24, 25, 27, 28, 29, 30...",33,33,7,4,30,18


In [43]:
import matplotlib.pyplot as plt
import matplotlib
# plt.figure(figsize=(25, 15))
# for site_id, data_by_site in weather_data_to_plot.groupby('site_id'):
#     by_site_by_hour = data_by_site.groupby('hour').mean()
#     ax = plt.subplot(4, 4, count)
#     plt.plot(by_site_by_hour.index,by_site_by_hour['air_temperature'],'xb-')
#     ax.set_title('site: '+str(site_id))
#     count += 1
# plt.tight_layout()
# plt.show()
# plt.scatter(range(click_log.shape[0]), click_log['click_times'])
# plt.show()

a = click_log.groupby(['click_times']).size()
a

click_times
1      60122248
2       3214136
3        199052
4         94546
5         17640
6          8249
7          3809
8          3673
9          1327
10          856
11          562
12          411
13          300
14          251
15          172
16          152
17          105
18          103
19           55
20           83
21           62
22           42
23           29
24           30
25           28
26           30
27           19
28           16
29           28
30           24
         ...   
72            3
73            2
76            5
77            2
79            3
80            3
82            1
84            2
87            2
88            1
90            1
94            1
96            1
97            2
103           2
106           1
108           1
113           1
119           1
120           1
121           1
122           1
124           1
127           2
133           1
146           1
152           1
158           1
167           1
185           1
Length: 94, 

In [49]:
np.percentile(click_log['click_times'], 99.99)

8.0